In [1]:
!pip install numpy pandas tensorflow scikit-learn matplotlib

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
dataset = pd.read_csv('Exp_Mn_Mw_Value.txt', sep='\t')
X = dataset.iloc[:, 1:5].values   # columns 1–4: Factor A–D
y = dataset.iloc[:, 5:7].values   # columns 5–6: Responses Mn, Mw
dataset.head()

,Run,Factor A,Factor B,Factor C,Factor D,Response 1 (Experimental),Response 2 (Experimental)
0,1,110,7,50,10,1127.19,1321.65
1,2,85,13,50,10,1024.97,1339.35
2,3,101,1,500,60,1950.00,2878.90
3,4,101,1,500,60,2223.17,2989.00
4,5,50,10,50,10,1845.60,2690.50


In [5]:
#normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_normalized = scaler_X.fit_transform(X)
y_normalized = scaler_y.fit_transform(y)

In [6]:
# Split data into 60% train, 20% validation, 20% test
# First split: 60% train, 40% temp (which will become 20% val + 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_normalized, y_normalized, test_size=0.4, random_state=42
)

# Second split: split the 40% temp into 20% validation and 20% test
# Since temp is 40% of total, splitting it 50/50 gives us 20%/20% of total
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)
# Print the sizes to verify
print(f"\nDataset split:")
print(f"Total samples: {len(X_normalized)}")
print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X_normalized)*100:.1f}%)")
print(f"Validation set: {X_val.shape[0]} samples ({X_val.shape[0]/len(X_normalized)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X_normalized)*100:.1f}%)")

print(f"\nFeature shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")

print(f"\nTarget shapes:")
print(f"y_train: {y_train.shape}")
print(f"y_val: {y_val.shape}")
print(f"y_test: {y_test.shape}")


Dataset split:
Total samples: 25
Training set: 15 samples (60.0%)
Validation set: 5 samples (20.0%)
Test set: 5 samples (20.0%)

Feature shapes:
X_train: (15, 4)
X_val: (5, 4)
X_test: (5, 4)

Target shapes:
y_train: (15, 2)
y_val: (5, 2)
y_test: (5, 2)


In [ ]:
#ann

print("NEURAL NETWORK ANALYSIS")


def create_neural_network(neurons, activation_function):
    """
    Create a simple neural network
    - neurons: number of neurons in hidden layer
    - activation_function: 'relu', 'tanh', 'sigmoid'
    """
    model = Sequential()
    
    # Input layer - we have 4 features (Factor A, B, C, D)
    model.add(Dense(neurons, input_shape=(4,), activation=activation_function))
    
    # Output layer - we predict 2 values (Mn and Mw)
    model.add(Dense(2, activation='linear'))  # linear for regression
    
    # Compile the model (set up for training)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

# Test different settings
neuron_counts = [8, 13, 20]  # different network sizes
activation_functions = ['relu', 'tanh', 'sigmoid', 'linear']  # different activation functions
# Note: 'linear' is the same as 'purelin' in TensorFlow/Keras

# Store all results
all_results = {}

print("Testing different neural network configurations...")


# Test each combination
for activation in activation_functions:
    print(f"Testing activation function: {activation.upper()}")

    for neurons in neuron_counts:
        print(f"  → Testing {neurons} neurons with {activation} activation...")
        
        # Set up 5-fold cross validation
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        r2_scores = []  # store R² score for each fold
        
        fold_number = 1
        
        # Do cross validation
        for train_idx, val_idx in kfold.split(X_train):
            # Split training data into train and validation for this fold
            X_train_fold = X_train[train_idx]
            X_val_fold = X_train[val_idx]
            y_train_fold = y_train[train_idx]
            y_val_fold = y_train[val_idx]
            
            # Create a new model for this fold
            model = create_neural_network(neurons, activation)
            
            # Set up early stopping to prevent overfitting
            early_stopping = EarlyStopping(
                monitor='val_loss',     # watch validation loss
                patience=20,            # stop if no improvement for 20 epochs
                restore_best_weights=True  # use best weights, not last
            )
            
            # Train the model
            model.fit(
                X_train_fold, y_train_fold,
                validation_data=(X_val_fold, y_val_fold),
                epochs=200,             # maximum training rounds
                batch_size=8,           # process 8 samples at a time
                verbose=0,              # don't print training progress
                callbacks=[early_stopping]
            )
            
            # Make predictions on validation set
            y_pred_fold = model.predict(X_val_fold, verbose=0)
            
            # Calculate how good the predictions are (R² score)
            r2 = r2_score(y_val_fold, y_pred_fold)
            r2_scores.append(r2)
            
            print(f"     Fold {fold_number}: R² = {r2:.4f}")
            fold_number += 1
        
        # Calculate average performance across all folds
        avg_r2 = np.mean(r2_scores)
        std_r2 = np.std(r2_scores)
        
        # Store results
        config_name = f"{activation}_{neurons}neurons"
        all_results[config_name] = {
            'activation': activation,
            'neurons': neurons,
            'avg_r2': avg_r2,
            'std_r2': std_r2,
            'all_scores': r2_scores
        }
        
        print(f"     → Average R²: {avg_r2:.4f} ± {std_r2:.4f}")
        print()
    
    print()

# Find the best configuration

print("RESULTS SUMMARY")


print("All configurations tested:")
print()

# Sort results by average R² score (best first)
sorted_results = sorted(all_results.items(), key=lambda x: x[1]['avg_r2'], reverse=True)

for i, (config_name, results) in enumerate(sorted_results):
    rank = i + 1
    activation = results['activation']
    neurons = results['neurons']
    avg_r2 = results['avg_r2']
    std_r2 = results['std_r2']
    
    print(f"{rank}. {activation.upper()} with {neurons} neurons: R² = {avg_r2:.4f} ± {std_r2:.4f}")

# Get the best configuration
best_config = sorted_results[0][1]
best_activation = best_config['activation']
best_neurons = best_config['neurons']
best_avg_r2 = best_config['avg_r2']

print()
print(" BEST CONFIGURATION:")
print(f"   Activation function: {best_activation.upper()}")
print(f"   Number of neurons: {best_neurons}")
print(f"   Average R²: {best_avg_r2:.4f}")

# Train final model with best configuration

print("TRAINING FINAL MODEL")


print(f"Training final model with {best_activation} activation and {best_neurons} neurons...")

# Create and train the final model
final_model = create_neural_network(best_neurons, best_activation)

# Use validation set for early stopping in final training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,  # a bit more patience for final model
    restore_best_weights=True
)

# Train on full training set, validate on validation set
history = final_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=300,
    batch_size=8,
    verbose=1,  # show progress for final training
    callbacks=[early_stopping]
)

# Test the final model
print("\nTesting final model on test set...")
y_pred_test = final_model.predict(X_test, verbose=0)
final_r2 = r2_score(y_test, y_pred_test)

print(f"\n FINAL RESULTS:")
print(f"   Test set R²: {final_r2:.4f}")

# Calculate individual R² for each output (Mn and Mw)
r2_mn = r2_score(y_test[:, 0], y_pred_test[:, 0])  # First output (Mn)
r2_mw = r2_score(y_test[:, 1], y_pred_test[:, 1])  # Second output (Mw)

print(f"   R² for Mn: {r2_mn:.4f}")
print(f"   R² for Mw: {r2_mw:.4f}")


# DETAILED COMPARISON SECTION

print("DETAILED ACTIVATION FUNCTION COMPARISON")


# Group results by activation function
activation_comparison = {}
for config_name, results in all_results.items():
    activation = results['activation']
    if activation not in activation_comparison:
        activation_comparison[activation] = []
    activation_comparison[activation].append(results)

print("Comparing each activation function across all neuron counts:\n")

for activation in ['relu', 'tanh', 'sigmoid', 'linear']:
    if activation in activation_comparison:
        print(f" {activation.upper()} ACTIVATION FUNCTION:")
        print("-" * 40)
        
        configs = activation_comparison[activation]
        # Sort by neurons for clear display
        configs_sorted = sorted(configs, key=lambda x: x['neurons'])
        
        best_for_this_activation = max(configs, key=lambda x: x['avg_r2'])
        
        for config in configs_sorted:
            neurons = config['neurons']
            avg_r2 = config['avg_r2']
            std_r2 = config['std_r2']

NEURAL NETWORK ANALYSIS
Testing different neural network configurations...
Testing activation function: RELU
  → Testing 8 neurons with relu activation...


C:\Users\brian\PycharmProjects\PythonProject1\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


     Fold 1: R² = -0.4927


C:\Users\brian\PycharmProjects\PythonProject1\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


     Fold 2: R² = -0.2591


...

C:\Users\brian\PycharmProjects\PythonProject1\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


     Fold 1: R² = -7.0743


C:\Users\brian\PycharmProjects\PythonProject1\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


     Fold 2: R² = -0.4039


C:\Users\brian\PycharmProjects\PythonProject1\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
#svm
# Additional imports for SVM analysis
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
print("SVM libraries imported successfully!")
print("Training SVM models...")

# Define parameter grids for hyperparameter tuning
param_grid = {
    'estimator__C': [0.1, 1, 10, 100],
    'estimator__kernel': ['linear', 'rbf', 'poly'],
    'estimator__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'estimator__epsilon': [0.01, 0.1, 0.2]
}

# Create SVM regressor for multi-output
svm_regressor = MultiOutputRegressor(SVR())

# Perform grid search with cross-validation
print("Performing hyperparameter tuning...")
grid_search = GridSearchCV(
    svm_regressor, 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# Fit the model
grid_search.fit(X_train_scaled, y_train_scaled)

# Get best model
best_svm = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {-grid_search.best_score_:.4f}")

# Train the best model
best_svm.fit(X_train_scaled, y_train_scaled)
print("Making predictions...")

# Make predictions on scaled data
y_train_pred_scaled = best_svm.predict(X_train_scaled)
y_val_pred_scaled = best_svm.predict(X_val_scaled)
y_test_pred_scaled = best_svm.predict(X_test_scaled)

# Transform predictions back to original scale
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled)
y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled)
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)

print("Predictions completed!")
print(f"Training predictions shape: {y_train_pred.shape}")
print(f"Validation predictions shape: {y_val_pred.shape}")
print(f"Test predictions shape: {y_test_pred.shape}")
def calculate_metrics(y_true, y_pred, set_name):
    """Calculate regression metrics for each output"""
    metrics = {}
    
    for i, response in enumerate(['Response_1', 'Response_2']):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        r2 = r2_score(y_true[:, i], y_pred[:, i])
        
        metrics[f'{response}'] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2
        }
    
    return metrics

# Calculate metrics for all sets
train_metrics = calculate_metrics(y_train, y_train_pred, 'Training')
val_metrics = calculate_metrics(y_val, y_val_pred, 'Validation')
test_metrics = calculate_metrics(y_test, y_test_pred, 'Test')

# Print metrics
print("SVM Performance Metrics:")
print("="*50)

for set_name, metrics in [('Training', train_metrics), ('Validation', val_metrics), ('Test', test_metrics)]:
    print(f"\n{set_name} Set:")
    for response, scores in metrics.items():
        print(f"  {response}:")
        for metric, value in scores.items():
            print(f"    {metric}: {value:.4f}")
            def create_comparison_table(y_true, y_pred, set_name):
    """Create a comparison table of actual vs predicted values"""
    
    # Create DataFrame
    comparison_df = pd.DataFrame({
        'Actual_Response_1': y_true[:, 0],
        'Predicted_Response_1': y_pred[:, 0],
        'Error_Response_1': y_true[:, 0] - y_pred[:, 0],
        'Actual_Response_2': y_true[:, 1],
        'Predicted_Response_2': y_pred[:, 1],
        'Error_Response_2': y_true[:, 1] - y_pred[:, 1],
    })
    
    # Add absolute percentage error
    comparison_df['APE_Response_1'] = np.abs(comparison_df['Error_Response_1'] / comparison_df['Actual_Response_1']) * 100
    comparison_df['APE_Response_2'] = np.abs(comparison_df['Error_Response_2'] / comparison_df['Actual_Response_2']) * 100
    
    return comparison_df

# Create comparison tables for all sets
train_comparison = create_comparison_table(y_train, y_train_pred, 'Training')
val_comparison = create_comparison_table(y_val, y_val_pred, 'Validation')
test_comparison = create_comparison_table(y_test, y_test_pred, 'Test')

# Display first 10 rows of each table
print("Training Set - Predicted vs Actual (First 10 rows):")
print(train_comparison.head(10).round(4))

print("\nValidation Set - Predicted vs Actual (First 10 rows):")
print(val_comparison.head(10).round(4))

print("\nTest Set - Predicted vs Actual (First 10 rows):")
print(test_comparison.head(10).round(4))

# Summary statistics
print("\nSummary Statistics:")
print("="*50)
for name, df in [('Training', train_comparison), ('Validation', val_comparison), ('Test', test_comparison)]:
    print(f"\n{name} Set:")
    print(f"  Mean APE Response 1: {df['APE_Response_1'].mean():.2f}%")
    print(f"  Mean APE Response 2: {df['APE_Response_2'].mean():.2f}%")
    print(f"  Max APE Response 1: {df['APE_Response_1'].max():.2f}%")
    print(f"  Max APE Response 2: {df['APE_Response_2'].max():.2f}%")
    plt.style.use('default')
fig = plt.figure(figsize=(20, 16))

# Color scheme for different sets
colors = {'Training': '#1f77b4', 'Validation': '#ff7f0e', 'Test': '#2ca02c'}
sets_data = {
    'Training': (y_train, y_train_pred, train_comparison),
    'Validation': (y_val, y_val_pred, val_comparison),
    'Test': (y_test, y_test_pred, test_comparison)
}

# 1. Predicted vs Actual Scatter Plots for Response 1
plt.subplot(4, 3, 1)
for set_name, (y_true, y_pred, _) in sets_data.items():
    plt.scatter(y_true[:, 0], y_pred[:, 0], alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

# Perfect prediction line
min_val = min([y_train[:, 0].min(), y_val[:, 0].min(), y_test[:, 0].min()])
max_val = max([y_train[:, 0].max(), y_val[:, 0].max(), y_test[:, 0].max()])
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Response 1')
plt.ylabel('Predicted Response 1')
plt.title('SVM: Predicted vs Actual - Response 1')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Predicted vs Actual Scatter Plots for Response 2
plt.subplot(4, 3, 2)
for set_name, (y_true, y_pred, _) in sets_data.items():
    plt.scatter(y_true[:, 1], y_pred[:, 1], alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

min_val = min([y_train[:, 1].min(), y_val[:, 1].min(), y_test[:, 1].min()])
max_val = max([y_train[:, 1].max(), y_val[:, 1].max(), y_test[:, 1].max()])
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Response 2')
plt.ylabel('Predicted Response 2')
plt.title('SVM: Predicted vs Actual - Response 2')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. R² Score Comparison
plt.subplot(4, 3, 3)
r2_scores_resp1 = [train_metrics['Response_1']['R²'], val_metrics['Response_1']['R²'], test_metrics['Response_1']['R²']]
r2_scores_resp2 = [train_metrics['Response_2']['R²'], val_metrics['Response_2']['R²'], test_metrics['Response_2']['R²']]

x_pos = np.arange(len(sets_data.keys()))
width = 0.35

plt.bar(x_pos - width/2, r2_scores_resp1, width, label='Response 1', alpha=0.8)
plt.bar(x_pos + width/2, r2_scores_resp2, width, label='Response 2', alpha=0.8)

plt.xlabel('Dataset')
plt.ylabel('R² Score')
plt.title('SVM: R² Score Comparison')
plt.xticks(x_pos, list(sets_data.keys()))
plt.legend()
plt.grid(True, alpha=0.3)

# 4. RMSE Comparison
plt.subplot(4, 3, 4)
rmse_scores_resp1 = [train_metrics['Response_1']['RMSE'], val_metrics['Response_1']['RMSE'], test_metrics['Response_1']['RMSE']]
rmse_scores_resp2 = [train_metrics['Response_2']['RMSE'], val_metrics['Response_2']['RMSE'], test_metrics['Response_2']['RMSE']]

plt.bar(x_pos - width/2, rmse_scores_resp1, width, label='Response 1', alpha=0.8)
plt.bar(x_pos + width/2, rmse_scores_resp2, width, label='Response 2', alpha=0.8)

plt.xlabel('Dataset')
plt.ylabel('RMSE')
plt.title('SVM: RMSE Comparison')
plt.xticks(x_pos, list(sets_data.keys()))
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Residuals Plot for Response 1
plt.subplot(4, 3, 5)
for set_name, (y_true, y_pred, _) in sets_data.items():
    residuals = y_true[:, 0] - y_pred[:, 0]
    plt.scatter(y_pred[:, 0], residuals, alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Predicted Response 1')
plt.ylabel('Residuals')
plt.title('SVM: Residuals Plot - Response 1')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Residuals Plot for Response 2
plt.subplot(4, 3, 6)
for set_name, (y_true, y_pred, _) in sets_data.items():
    residuals = y_true[:, 1] - y_pred[:, 1]
    plt.scatter(y_pred[:, 1], residuals, alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Predicted Response 2')
plt.ylabel('Residuals')
plt.title('SVM: Residuals Plot - Response 2')
plt.legend()
plt.grid(True, alpha=0.3)

# 7. Prediction Error Distribution - Response 1
plt.subplot(4, 3, 7)
for set_name, (_, _, comparison_df) in sets_data.items():
    plt.hist(comparison_df['Error_Response_1'], bins=20, alpha=0.6, 
             label=f'{set_name}', density=True, color=colors[set_name])

plt.xlabel('Prediction Error')
plt.ylabel('Density')
plt.title('SVM: Error Distribution - Response 1')
plt.legend()
plt.grid(True, alpha=0.3)

# 8. Prediction Error Distribution - Response 2
plt.subplot(4, 3, 8)
for set_name, (_, _, comparison_df) in sets_data.items():
    plt.hist(comparison_df['Error_Response_2'], bins=20, alpha=0.6, 
             label=f'{set_name}', density=True, color=colors[set_name])

plt.xlabel('Prediction Error')
plt.ylabel('Density')
plt.title('SVM: Error Distribution - Response 2')
plt.legend()
plt.grid(True, alpha=0.3)

# 9. Absolute Percentage Error Box Plot
plt.subplot(4, 3, 9)
ape_data = []
labels = []

for set_name, (_, _, comparison_df) in sets_data.items():
    ape_data.extend([comparison_df['APE_Response_1'].values, comparison_df['APE_Response_2'].values])
    labels.extend([f'{set_name}\nResp 1', f'{set_name}\nResp 2'])

plt.boxplot(ape_data, labels=labels)
plt.ylabel('Absolute Percentage Error (%)')
plt.title('SVM: APE Distribution Across Sets')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 10. Feature Importance (if available from linear kernel)
plt.subplot(4, 3, 10)
try:
    # Try to get feature importance for Response 1
    if hasattr(best_svm.estimators_[0], 'coef_'):
        importance_resp1 = np.abs(best_svm.estimators_[0].coef_[0])
        importance_resp2 = np.abs(best_svm.estimators_[1].coef_[0])
        
        features = ['Factor A', 'Factor B', 'Factor C', 'Factor D']
        x_pos = np.arange(len(features))
        
        plt.bar(x_pos - width/2, importance_resp1, width, label='Response 1', alpha=0.8)
        plt.bar(x_pos + width/2, importance_resp2, width, label='Response 2', alpha=0.8)
        
        plt.xlabel('Features')
        plt.ylabel('Importance (|Coefficient|)')
        plt.title('SVM: Feature Importance')
        plt.xticks(x_pos, features)
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'Feature importance not available\nfor non-linear kernel', 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('SVM: Feature Importance')
except:
    plt.text(0.5, 0.5, 'Feature importance calculation failed', 
            ha='center', va='center', transform=plt.gca().transAxes)
    plt.title('SVM: Feature Importance')

plt.grid(True, alpha=0.3)

# 11. Model Performance Summary
plt.subplot(4, 3, 11)
plt.axis('off')
summary_text = f"""
SVM Model Performance Summary

Best Parameters:
{str(grid_search.best_params_).replace(',', ',\n')}

Test Set Performance:
Response 1: R² = {test_metrics['Response_1']['R²']:.4f}
Response 2: R² = {test_metrics['Response_2']['R²']:.4f}

Response 1: RMSE = {test_metrics['Response_1']['RMSE']:.4f}
Response 2: RMSE = {test_metrics['Response_2']['RMSE']:.4f}

Mean APE:
Response 1: {test_comparison['APE_Response_1'].mean():.2f}%
Response 2: {test_comparison['APE_Response_2'].mean():.2f}%
"""
plt.text(0.05, 0.95, summary_text, transform=plt.gca().transAxes, 
         verticalalignment='top', fontsize=10, fontfamily='monospace')

# 12. Cross-validation scores visualization
plt.subplot(4, 3, 12)
cv_scores = cross_val_score(best_svm, X_train_scaled, y_train_scaled, cv=5, 
                           scoring='neg_mean_squared_error')
cv_scores = -cv_scores  # Convert back to positive

plt.bar(range(1, 6), cv_scores, alpha=0.8)
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--', 
            label=f'Mean: {cv_scores.mean():.4f}')
plt.xlabel('Fold')
plt.ylabel('MSE')
plt.title('SVM: Cross-Validation Scores')
plt.legend()
plt.grid(True, alpha=0.3)
plt.style.use('default')
fig = plt.figure(figsize=(20, 16))

# Color scheme for different sets
colors = {'Training': '#1f77b4', 'Validation': '#ff7f0e', 'Test': '#2ca02c'}
sets_data = {
    'Training': (y_train, y_train_pred, train_comparison),
    'Validation': (y_val, y_val_pred, val_comparison),
    'Test': (y_test, y_test_pred, test_comparison)
}

# 1. Predicted vs Actual Scatter Plots for Response 1
plt.subplot(4, 3, 1)
for set_name, (y_true, y_pred, _) in sets_data.items():
    plt.scatter(y_true[:, 0], y_pred[:, 0], alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

# Perfect prediction line
min_val = min([y_train[:, 0].min(), y_val[:, 0].min(), y_test[:, 0].min()])
max_val = max([y_train[:, 0].max(), y_val[:, 0].max(), y_test[:, 0].max()])
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Response 1')
plt.ylabel('Predicted Response 1')
plt.title('SVM: Predicted vs Actual - Response 1')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Predicted vs Actual Scatter Plots for Response 2
plt.subplot(4, 3, 2)
for set_name, (y_true, y_pred, _) in sets_data.items():
    plt.scatter(y_true[:, 1], y_pred[:, 1], alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

min_val = min([y_train[:, 1].min(), y_val[:, 1].min(), y_test[:, 1].min()])
max_val = max([y_train[:, 1].max(), y_val[:, 1].max(), y_test[:, 1].max()])
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Response 2')
plt.ylabel('Predicted Response 2')
plt.title('SVM: Predicted vs Actual - Response 2')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. R² Score Comparison
plt.subplot(4, 3, 3)
r2_scores_resp1 = [train_metrics['Response_1']['R²'], val_metrics['Response_1']['R²'], test_metrics['Response_1']['R²']]
r2_scores_resp2 = [train_metrics['Response_2']['R²'], val_metrics['Response_2']['R²'], test_metrics['Response_2']['R²']]

x_pos = np.arange(len(sets_data.keys()))
width = 0.35

plt.bar(x_pos - width/2, r2_scores_resp1, width, label='Response 1', alpha=0.8)
plt.bar(x_pos + width/2, r2_scores_resp2, width, label='Response 2', alpha=0.8)

plt.xlabel('Dataset')
plt.ylabel('R² Score')
plt.title('SVM: R² Score Comparison')
plt.xticks(x_pos, list(sets_data.keys()))
plt.legend()
plt.grid(True, alpha=0.3)

# 4. RMSE Comparison
plt.subplot(4, 3, 4)
rmse_scores_resp1 = [train_metrics['Response_1']['RMSE'], val_metrics['Response_1']['RMSE'], test_metrics['Response_1']['RMSE']]
rmse_scores_resp2 = [train_metrics['Response_2']['RMSE'], val_metrics['Response_2']['RMSE'], test_metrics['Response_2']['RMSE']]

plt.bar(x_pos - width/2, rmse_scores_resp1, width, label='Response 1', alpha=0.8)
plt.bar(x_pos + width/2, rmse_scores_resp2, width, label='Response 2', alpha=0.8)

plt.xlabel('Dataset')
plt.ylabel('RMSE')
plt.title('SVM: RMSE Comparison')
plt.xticks(x_pos, list(sets_data.keys()))
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Residuals Plot for Response 1
plt.subplot(4, 3, 5)
for set_name, (y_true, y_pred, _) in sets_data.items():
    residuals = y_true[:, 0] - y_pred[:, 0]
    plt.scatter(y_pred[:, 0], residuals, alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Predicted Response 1')
plt.ylabel('Residuals')
plt.title('SVM: Residuals Plot - Response 1')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Residuals Plot for Response 2
plt.subplot(4, 3, 6)
for set_name, (y_true, y_pred, _) in sets_data.items():
    residuals = y_true[:, 1] - y_pred[:, 1]
    plt.scatter(y_pred[:, 1], residuals, alpha=0.6, label=f'{set_name}', 
                color=colors[set_name], s=50)

plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Predicted Response 2')
plt.ylabel('Residuals')
plt.title('SVM: Residuals Plot - Response 2')
plt.legend()
plt.grid(True, alpha=0.3)

# 7. Prediction Error Distribution - Response 1
plt.subplot(4, 3, 7)
for set_name, (_, _, comparison_df) in sets_data.items():
    plt.hist(comparison_df['Error_Response_1'], bins=20, alpha=0.6, 
             label=f'{set_name}', density=True, color=colors[set_name])

plt.xlabel('Prediction Error')
plt.ylabel('Density')
plt.title('SVM: Error Distribution - Response 1')
plt.legend()
plt.grid(True, alpha=0.3)

# 8. Prediction Error Distribution - Response 2
plt.subplot(4, 3, 8)
for set_name, (_, _, comparison_df) in sets_data.items():
    plt.hist(comparison_df['Error_Response_2'], bins=20, alpha=0.6, 
             label=f'{set_name}', density=True, color=colors[set_name])

plt.xlabel('Prediction Error')
plt.ylabel('Density')
plt.title('SVM: Error Distribution - Response 2')
plt.legend()
plt.grid(True, alpha=0.3)

# 9. Absolute Percentage Error Box Plot
plt.subplot(4, 3, 9)
ape_data = []
labels = []

for set_name, (_, _, comparison_df) in sets_data.items():
    ape_data.extend([comparison_df['APE_Response_1'].values, comparison_df['APE_Response_2'].values])
    labels.extend([f'{set_name}\nResp 1', f'{set_name}\nResp 2'])

plt.boxplot(ape_data, labels=labels)
plt.ylabel('Absolute Percentage Error (%)')
plt.title('SVM: APE Distribution Across Sets')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 10. Feature Importance (if available from linear kernel)
plt.subplot(4, 3, 10)
try:
    # Try to get feature importance for Response 1
    if hasattr(best_svm.estimators_[0], 'coef_'):
        importance_resp1 = np.abs(best_svm.estimators_[0].coef_[0])
        importance_resp2 = np.abs(best_svm.estimators_[1].coef_[0])
        
        features = ['Factor A', 'Factor B', 'Factor C', 'Factor D']
        x_pos = np.arange(len(features))
        
        plt.bar(x_pos - width/2, importance_resp1, width, label='Response 1', alpha=0.8)
        plt.bar(x_pos + width/2, importance_resp2, width, label='Response 2', alpha=0.8)
        
        plt.xlabel('Features')
        plt.ylabel('Importance (|Coefficient|)')
        plt.title('SVM: Feature Importance')
        plt.xticks(x_pos, features)
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'Feature importance not available\nfor non-linear kernel', 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('SVM: Feature Importance')
except:
    plt.text(0.5, 0.5, 'Feature importance calculation failed', 
            ha='center', va='center', transform=plt.gca().transAxes)
    plt.title('SVM: Feature Importance')

plt.grid(True, alpha=0.3)

# 11. Model Performance Summary
plt.subplot(4, 3, 11)
plt.axis('off')
summary_text = f"""
SVM Model Performance Summary

Best Parameters:
{str(grid_search.best_params_).replace(',', ',\n')}

Test Set Performance:
Response 1: R² = {test_metrics['Response_1']['R²']:.4f}
Response 2: R² = {test_metrics['Response_2']['R²']:.4f}

Response 1: RMSE = {test_metrics['Response_1']['RMSE']:.4f}
Response 2: RMSE = {test_metrics['Response_2']['RMSE']:.4f}

Mean APE:
Response 1: {test_comparison['APE_Response_1'].mean():.2f}%
Response 2: {test_comparison['APE_Response_2'].mean():.2f}%
"""
plt.text(0.05, 0.95, summary_text, transform=plt.gca().transAxes, 
         verticalalignment='top', fontsize=10, fontfamily='monospace')

# 12. Cross-validation scores visualization
plt.subplot(4, 3, 12)
cv_scores = cross_val_score(best_svm, X_train_scaled, y_train_scaled, cv=5, 
                           scoring='neg_mean_squared_error')
cv_scores = -cv_scores  # Convert back to positive

plt.bar(range(1, 6), cv_scores, alpha=0.8)
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--', 
            label=f'Mean: {cv_scores.mean():.4f}')
plt.xlabel('Fold')
plt.ylabel('MSE')
plt.title('SVM: Cross-Validation Scores')
plt.legend()
plt.grid(True, alpha=0.3)

IndentationError: expected an indented block after function definition on line 91 (2257375561.py, line 92)

In [ ]:
# First, we need to convert predictions back to original scale for plotting
# (since we normalized the data earlier)

print("Preparing data for visualization...")

# Convert test predictions back to original scale
y_pred_nn_original = scaler_y.inverse_transform(y_pred_test)
y_test_original = scaler_y.inverse_transform(y_test)

# Calculate R² for individual responses (Mn and Mw)
nn_r2_resp1 = r2_score(y_test_original[:, 0], y_pred_nn_original[:, 0])  # Response 1 (Mn)
nn_r2_resp2 = r2_score(y_test_original[:, 1], y_pred_nn_original[:, 1])  # Response 2 (Mw)

# Get all R² values for statistics (fix for the error)
all_r2_values = [results['avg_r2'] for results in all_results.values()]

# Create comprehensive plots
plt.figure(figsize=(18, 12))

# Plot 1: Activation Function Comparison
plt.subplot(3, 4, 1)
activation_names = []
best_r2_scores = []
best_neurons_list = []

for activation in ['relu', 'tanh', 'sigmoid', 'linear']:
    if activation in activation_comparison:
        best_config = max(activation_comparison[activation], key=lambda x: x['avg_r2'])
        activation_names.append(activation.upper())
        best_r2_scores.append(best_config['avg_r2'])
        best_neurons_list.append(best_config['neurons'])

colors = ['red', 'blue', 'green', 'orange']
bars = plt.bar(activation_names, best_r2_scores, color=colors, alpha=0.7)
plt.title('Best R² Score by Activation Function')
plt.xlabel('Activation Function')
plt.ylabel('Best R² Score')
plt.ylim(0, 1)

# Add value labels on bars
for i, (v, neurons) in enumerate(zip(best_r2_scores, best_neurons_list)):
    plt.text(i, v + 0.02, f'{v:.3f}\n({neurons} neurons)', ha='center', fontsize=9)

# Plot 2: Neuron Count Comparison for Best Activation
plt.subplot(3, 4, 2)
best_activation_configs = activation_comparison[best_activation]
neuron_counts_sorted = sorted([config['neurons'] for config in best_activation_configs])
r2_for_neurons = []

for neurons in neuron_counts_sorted:
    config = next(c for c in best_activation_configs if c['neurons'] == neurons)
    r2_for_neurons.append(config['avg_r2'])

plt.bar([str(n) for n in neuron_counts_sorted], r2_for_neurons, 
        color='blue', alpha=0.7, capsize=5)
plt.title(f'{best_activation.upper()} Activation: R² vs Neurons')
plt.xlabel('Number of Neurons')
plt.ylabel('R² Score')
plt.ylim(0, 1)

for i, v in enumerate(r2_for_neurons):
    plt.text(i, v + 0.02, f'{v:.3f}', ha='center')

# Plot 3: Cross-validation stability comparison
plt.subplot(3, 4, 3)
std_devs = [results['std_r2'] for results in [max(activation_comparison[act], key=lambda x: x['avg_r2']) 
                                            for act in ['relu', 'tanh', 'sigmoid', 'linear'] 
                                            if act in activation_comparison]]
plt.bar(activation_names, std_devs, color=colors, alpha=0.7)
plt.title('Model Stability (Lower = More Stable)')
plt.xlabel('Activation Function')
plt.ylabel('Standard Deviation')

for i, v in enumerate(std_devs):
    plt.text(i, v + max(std_devs)*0.05, f'{v:.4f}', ha='center', fontsize=9)

# Plot 4: All configurations heatmap-style
plt.subplot(3, 4, 4)
# Create matrix for heatmap
activations = ['relu', 'tanh', 'sigmoid', 'linear']
neurons = [8, 13, 20]
r2_matrix = np.zeros((len(activations), len(neurons)))

for i, activation in enumerate(activations):
    if activation in activation_comparison:
        for config in activation_comparison[activation]:
            j = neurons.index(config['neurons'])
            r2_matrix[i, j] = config['avg_r2']

im = plt.imshow(r2_matrix, cmap='viridis', aspect='auto')
plt.colorbar(im, label='R² Score')
plt.title('R² Scores: All Configurations')
plt.xlabel('Number of Neurons')
plt.ylabel('Activation Function')
plt.xticks(range(len(neurons)), neurons)
plt.yticks(range(len(activations)), [a.upper() for a in activations])

# Add text annotations
for i in range(len(activations)):
    for j in range(len(neurons)):
        if r2_matrix[i, j] > 0:
            plt.text(j, i, f'{r2_matrix[i,j]:.3f}', ha='center', va='center', 
                    color='white' if r2_matrix[i,j] < 0.5 else 'black')

# Plot 5: Neural Network Response 1 (Mn) - Actual vs Predicted
plt.subplot(3, 4, 5)
plt.scatter(y_test_original[:, 0], y_pred_nn_original[:, 0], alpha=0.7, color='blue')
plt.plot([y_test_original[:, 0].min(), y_test_original[:, 0].max()], 
         [y_test_original[:, 0].min(), y_test_original[:, 0].max()], 'r--', lw=2)
plt.xlabel('Actual Mn')
plt.ylabel('Predicted Mn')
plt.title(f'Neural Network - Mn (Response 1)\nR² = {nn_r2_resp1:.3f}')
plt.grid(True, alpha=0.3)

# Plot 6: Neural Network Response 2 (Mw) - Actual vs Predicted
plt.subplot(3, 4, 6)
plt.scatter(y_test_original[:, 1], y_pred_nn_original[:, 1], alpha=0.7, color='blue')
plt.plot([y_test_original[:, 1].min(), y_test_original[:, 1].max()], 
         [y_test_original[:, 1].min(), y_test_original[:, 1].max()], 'r--', lw=2)
plt.xlabel('Actual Mw')
plt.ylabel('Predicted Mw')
plt.title(f'Neural Network - Mw (Response 2)\nR² = {nn_r2_resp2:.3f}')
plt.grid(True, alpha=0.3)

# Plot 7: Residuals plot for Response 1 (Mn)
plt.subplot(3, 4, 7)
residuals_1 = y_test_original[:, 0] - y_pred_nn_original[:, 0]
plt.scatter(y_pred_nn_original[:, 0], residuals_1, alpha=0.7, color='blue')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Mn')
plt.ylabel('Residuals')
plt.title('Residuals Plot - Mn')
plt.grid(True, alpha=0.3)

# Plot 8: Residuals plot for Response 2 (Mw)
plt.subplot(3, 4, 8)
residuals_2 = y_test_original[:, 1] - y_pred_nn_original[:, 1]
plt.scatter(y_pred_nn_original[:, 1], residuals_2, alpha=0.7, color='blue')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Mw')
plt.ylabel('Residuals')
plt.title('Residuals Plot - Mw')
plt.grid(True, alpha=0.3)

# Plot 9: Best model summary
plt.subplot(3, 4, 9)
metrics = ['Overall R²', 'Mn R²', 'Mw R²']
values = [final_r2, nn_r2_resp1, nn_r2_resp2]
bars = plt.bar(metrics, values, color=['purple', 'blue', 'green'], alpha=0.7)
plt.title(f'Final Model Performance\n{best_activation.upper()} - {best_neurons} neurons')
plt.ylabel('R² Score')